In [4]:
import pandas as pd

# Cargar archivo CSV (ajusta la ruta si es necesario)
df = pd.read_csv("../../artifacts/configuration/csv/columns.csv", sep=";", encoding="latin1")

# Filtrar columnas necesarias y ordenar por TABLE_NAME y COLUMN_ID
df = df[['TABLE_NAME', 'COLUMN_NAME', 'NEW_DATA_TYPE', 'COLUMN_ID']]
df = df.sort_values(by=['TABLE_NAME', 'COLUMN_ID'])

# Mapeo de tipos a DBML
type_map = {
    'string': 'varchar',
    'timestamp': 'timestamp',
    'int': 'int',
    'bigint': 'bigint',
    'double': 'float',
    'boolean': 'boolean'
}

# Listado de tablas
all_tables = df['TABLE_NAME'].unique()

# Construcción del DBML
output = ""
for table in all_tables:
    table_df = df[df['TABLE_NAME'] == table]
    output += f"Table {table} {{\n"
    
    for _, row in table_df.iterrows():
        col_name = row['COLUMN_NAME']
        col_type = type_map.get(row['NEW_DATA_TYPE'].lower(), 'varchar')
        is_pk = row['COLUMN_ID'] == 0
        flags = []

        # Clave primaria
        if is_pk:
            flags.append("pk")

        # Clave foránea (solo si no es PK y es id_)
        if col_name.startswith("id_") and not is_pk:
            ref_key = col_name.replace("id_", "").upper()
            for other_table in all_tables:
                if other_table != table and ref_key in other_table.upper():
                    flags.append(f"ref: > {other_table}.{col_name}")
                    break  # usa solo la primera coincidencia válida

        flag_str = f" [{', '.join(flags)}]" if flags else ""
        output += f"  {col_name} {col_type}{flag_str}\n"
    
    output += "}\n\n"

# Guardar resultado
with open("output.dbml", "w", encoding="utf-8") as f:
    f.write(output)

print("✅ Archivo DBML generado como 'output.dbml'")


✅ Archivo DBML generado como 'output.dbml'
